In [ ]:
import pandas as pd

# Đọc dữ liệu từ file Parquet
track_credit_df = pd.read_parquet("data/track_credit_df.par")
tps_df = pd.read_parquet("data/tps.par")

# Hiển thị thông tin cơ bản của dataframe
track_credit_df
tps_df

In [ ]:
from sqlalchemy import Column, Integer, String, Float, DateTime, create_engine, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy.orm import sessionmaker

Base = declarative_base()

# Bảng track_credit
class TrackCredit(Base):
    __tablename__ = 'track_credit'

    id = Column(Integer, primary_key=True, autoincrement=True)
    track_id = Column(Integer, nullable=False)
    artist_id = Column(Integer, nullable=False)
    role_id = Column(Integer, nullable=False)
    created_at = Column(DateTime, nullable=False)
    updated_at = Column(DateTime, nullable=False)
    position = Column(Float, nullable=False)

    # Quan hệ với bảng TPS
    tps = relationship("TPS", back_populates="track_credit")

# Bảng tps
class TPS(Base):
    __tablename__ = 'tps'

    track_id = Column(Integer, ForeignKey('track_credit.track_id'), primary_key=True, nullable=False)
    user_id = Column(Integer, nullable=False)
    date = Column(DateTime, nullable=False)
    listen_count = Column(Integer, nullable=False)
    created_at = Column(DateTime, nullable=False)
    updated_at = Column(DateTime, nullable=False)

    # Quan hệ ngược lại với bảng TrackCredit
    track_credit = relationship("TrackCredit", back_populates="tps", lazy="dynamic")

# Tạo tất cả bảng dựa trên định nghĩa ORM
Base.metadata.create_all(engine)

# Tạo session để làm việc
Session = sessionmaker(bind=engine)
session = Session()

# JOIN hai bảng track_credit và tps theo track_id 
# Chọn các cột track_id, artist_id từ bảng track_credit 
# và các cột user_id, listen_count từ bảng tps
# limit 100 kết quả
join_query = session.query(TrackCredit.track_id, TrackCredit.artist_id, TPS.user_id, TPS.listen_count).join(TPS, TrackCredit.track_id == TPS.track_id).limit(100)

for row in join_query:
    print(f"Track ID: {row.track_id}, Artist ID: {row.artist_id}, User ID: {row.user_id}, Listen Count: {row.listen_count}")

# Đóng session
session.close()


In [ ]:
from sqlalchemy.sql import func, select, text

# Thống kê: Đếm số lượng bản ghi
count_query = session.query(func.count()).select_from(TrackCredit).scalar()
print(f"Tổng số bản ghi: {count_query}")

# Tìm giá trị lớn nhất và nhỏ nhất cho một cột (ví dụ cột 'age')
max_query = session.query(func.max(TPS.listen_count)).scalar()
min_query = session.query(func.min(TPS.listen_count)).scalar()
print(f"Giá trị lớn nhất: {max_query}, Giá trị nhỏ nhất: {min_query}")